# Step 4: Filling datagaps with IMDB API and final dataset creation

In [ ]:
import numpy as np
import pandas as pd
import ast

# import matplotlib.pyplot as plt
# import matplotlib.patches as mpatches

# Load Datasets

In [2]:
movies_master = pd.read_csv('data/MoviesOnStreamingPlatforms.csv')
#Source:https://www.kaggle.com/ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney
tv_shows_master = pd.read_csv('data/tv_shows.csv') 
emmy = pd.read_csv('data/the_emmy_awards.csv')
oscar = pd.read_csv('data/the_oscar_award.csv')

type_amazon= pd.read_csv('data/amazon_prime_titles.csv')
type_netflix = pd.read_csv('data/netflix_titles.csv')

#file created by Hyunwoo (data scraped from API)
main_alt = pd.read_csv('data/main.csv') 

In [3]:
type_netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [4]:
#checking the size of of all movies
movies_master.shape[0]

9515

In [5]:
# Select the columns required for the analyses & change the name of the Prime Video column to Amazon
cols = ['ID','Title', 'Year', 'Age', 'Rotten Tomatoes', 'Netflix', 'Prime Video']
movies_all = movies_master.loc[:,cols].rename(columns={'Prime Video': 'Amazon'})

In [6]:
# Drop titles that are not on Netflix or Amazon
movies = movies_all[(movies_all['Netflix']==1) | (movies_all['Amazon']==1)]

In [7]:
#Checking for the number of unique values for each columns.  
movies.nunique(axis=0)

ID                 7699
Title              7699
Year                102
Age                   5
Rotten Tomatoes      84
Netflix               2
Amazon                2
dtype: int64

In [8]:
#checking for non null values to identity columns that are can be potentially removed.
#the ratings column has a high number of null values, but this column is essential for our analysis.
#so will look to fill the null values with values from other sources

movies.isnull().sum()

ID                    0
Title                 0
Year                  0
Age                3702
Rotten Tomatoes       7
Netflix               0
Amazon                0
dtype: int64

In [9]:
movies.head()

,ID,Title,Year,Age,Rotten Tomatoes,Netflix,Amazon
0,1,The Irishman,2019,18+,98/100,1,0
1,2,Dangal,2016,7+,97/100,1,0
2,3,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0
3,4,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0
4,5,Roma,2018,18+,94/100,1,0


In [10]:
#uncomment if you want to see the the year range for which the titles are avaiable 
#sorted(movies.Year.unique())

In [11]:
#uncomment if you want to search for specific movies
#movies.loc[movies['Title'].str.contains('Barry', na=False)]

In [12]:
#note that that type is set to one for all rows. Change this to "tvshow"
tv_shows_master.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type
0,0,1,Breaking Bad,2008,18+,9.4/10,100/100,1,0,0,0,1
1,1,2,Stranger Things,2016,16+,8.7/10,96/100,1,0,0,0,1
2,2,3,Attack on Titan,2013,18+,9.0/10,95/100,1,1,0,0,1
3,3,4,Better Call Saul,2015,18+,8.8/10,94/100,1,0,0,0,1
4,4,5,Dark,2017,16+,8.8/10,93/100,1,0,0,0,1


In [13]:
# Select the columns required for the analyses & change the name of the Prime Video column to Amazon
# Set the type column to tvshows
tv_shows_master.drop(['Unnamed: 0','Hulu','Disney+', 'Type'], axis=1,inplace =True)
tv_shows_master.rename(columns={"Prime Video": "Amazon"}, inplace=True)
tv_shows = tv_shows_master[(tv_shows_master['Netflix']==1) | (tv_shows_master['Amazon']==1)]
tv_shows['Type'] ="tvshow"

C:\Users\kimh\AppData\Local\Temp/ipykernel_14956/4002319544.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_shows['Type'] ="tvshow"


In [14]:
#add Type column to movies dataset
movies['Type'] = np.nan
movies['IMDb'] = np.nan

C:\Users\kimh\AppData\Local\Temp/ipykernel_14956/4081343017.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['Type'] = np.nan
C:\Users\kimh\AppData\Local\Temp/ipykernel_14956/4081343017.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['IMDb'] = np.nan


In [15]:
#check the update
tv_shows.head()

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Amazon,Type
0,1,Breaking Bad,2008,18+,9.4/10,100/100,1,0,tvshow
1,2,Stranger Things,2016,16+,8.7/10,96/100,1,0,tvshow
2,3,Attack on Titan,2013,18+,9.0/10,95/100,1,0,tvshow
3,4,Better Call Saul,2015,18+,8.8/10,94/100,1,0,tvshow
4,5,Dark,2017,16+,8.8/10,93/100,1,0,tvshow


In [16]:
#check the number of rows
tv_shows.shape[0]

3696

In [17]:
tv_shows.isnull().sum()

ID                    0
Title                 0
Year                  0
Age                1678
IMDb                742
Rotten Tomatoes       0
Netflix               0
Amazon                0
Type                  0
dtype: int64

In [18]:
#merge movies and tvshows datasets
main = pd.concat([movies[tv_shows.columns], tv_shows])
main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11395 entries, 0 to 5303
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   ID               11395 non-null  int64 
 1   Title            11395 non-null  object
 2   Year             11395 non-null  int64 
 3   Age              6015 non-null   object
 4   IMDb             2954 non-null   object
 5   Rotten Tomatoes  11388 non-null  object
 6   Netflix          11395 non-null  int64 
 7   Amazon           11395 non-null  int64 
 8   Type             3696 non-null   object
dtypes: int64(4), object(5)
memory usage: 890.2+ KB


In [19]:
#Get unique movie titles
all_titles = main['Title'].unique()

In [20]:
#dowload file from https://datasets.imdbws.com/
title_info = pd.DataFrame()
chunksize = 10 ** 5
for chunk in pd.read_csv("data/title.basics.tsv", delimiter='\t', chunksize=chunksize):
    #title_info=title_info.append (chunk)
    title_info=title_info.append (chunk[chunk.originalTitle.isin(all_titles)])

C:\Users\kimh\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3377: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\kimh\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3377: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [21]:
all_tconst = title_info['tconst'].unique()

In [22]:
#download the ratings file from https://datasets.imdbws.com/
ratings = pd.DataFrame()
chunksize = 10 ** 5
for chunk in pd.read_csv("data/title.ratings.tsv", delimiter='\t', chunksize=chunksize):
    #title_info=title_info.append (chunk)
    ratings=ratings.append (chunk[chunk.tconst.isin(all_tconst)])

In [23]:
#performing checks on unique values. 
title_info.nunique(axis=0)

tconst            89816
titleType            10
primaryTitle       9535
originalTitle      9154
isAdult               4
startYear           198
endYear              73
runtimeMinutes      330
genres             1218
dtype: int64

In [24]:
ratings.head()

,tconst,averageRating,numVotes
177,tt0000181,6.8,58
217,tt0000245,6.2,48
370,tt0000565,6.3,576
414,tt0000644,5.5,63
429,tt0000668,7.1,50


In [25]:
title_info.isnull().sum()

tconst            0
titleType         0
primaryTitle      0
originalTitle     0
isAdult           0
startYear         0
endYear           0
runtimeMinutes    0
genres            0
dtype: int64

In [26]:
title_info.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
178,tt0000181,short,Cinderella,Cinderella,0,1898,\N,\N,"Fantasy,Short"
242,tt0000245,short,The Haunted House,The Haunted House,0,1899,\N,\N,"Comedy,Fantasy,Horror"
474,tt0000478,short,Behind the Scenes,Behind the Scenes,0,1904,\N,\N,"Comedy,Short"
561,tt0000565,short,The Night Before Christmas,The Night Before Christmas,0,1905,\N,9,"Animation,Family,Fantasy"
638,tt0000644,short,Behind the Scenes,Behind the Scenes,0,1908,\N,9,"Drama,Short"


In [27]:
#uncomment if you want to test movies in the title_info file
#title_info[(title_info['originalTitle'] == 'Cinderella') &(title_info['startYear'] == '1965')]


In [28]:
#uncomment if you want to compare movies from title and movies datasets
#main[main['Title'] == 'Cinderella']

In [29]:
#Replace first \N with nan value
title_info = title_info.replace('\\N',np.NaN)
title_info.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
178,tt0000181,short,Cinderella,Cinderella,0,1898,NaN,NaN,"Fantasy,Short"
242,tt0000245,short,The Haunted House,The Haunted House,0,1899,NaN,NaN,"Comedy,Fantasy,Horror"
474,tt0000478,short,Behind the Scenes,Behind the Scenes,0,1904,NaN,NaN,"Comedy,Short"
561,tt0000565,short,The Night Before Christmas,The Night Before Christmas,0,1905,NaN,9,"Animation,Family,Fantasy"
638,tt0000644,short,Behind the Scenes,Behind the Scenes,0,1908,NaN,9,"Drama,Short"


In [30]:
#Remove entries for years less than 1912 (since we know that movies dataset only has movies from 1912), 
#genres that are talkshows, and titles that are short, videos, tvepisodes, tvminiseries, videogames and tvspecials
#also removed tvepisode since they didn't seem relevent for our analysis. 
title_info_clean = title_info[((title_info['startYear'].astype(np.float)>=1912) 
                               & (title_info['genres'].str.contains('Talk-Show') == False)
                               & (title_info['titleType'].isin(['short','video','tvShort','tvEpisode', 'tvSpecial', 'tvMiniSeries', 'videoGame']) == False)
                              )]

C:\Users\kimh\AppData\Local\Temp/ipykernel_14956/340623367.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  title_info_clean = title_info[((title_info['startYear'].astype(np.float)>=1912)


In [31]:
#testing to check the titletypes available
title_info_clean.titleType.unique()

array(['movie', 'tvSeries', 'tvMovie'], dtype=object)

In [32]:
#movies['Year'].dtypes

In [33]:
#After removing the unwanted years, the duplicates reduced from 68716 to 50444
#but if you just check the unique values, there are about 28470
#uncomment to check unique values
#title_info_clean.loc[title_info_clean.originalTitle.duplicated(),['originalTitle']].originalTitle.unique().size
#title_info_clean.loc[title_info_clean.originalTitle.duplicated(),['originalTitle']].originalTitle.unique


In [34]:
#I checked a couple of them and the duplicates seem to be legitimate (Oliver Twist for example). Meaning some are movies that came in different years or shows
#The other point is that the movies database doesn't seem to be comprehesive enough. 
#also remove entries which are talk shows. 
#Strange that some of the movies are not in this file downloaded from IMDB: Along with the Gods: The Two Worlds, The First King 
#Though you can find IMDB ratings for them when you search for the movie names. 

#uncomment to test movies with names that are in movies database and not in IMDB database
#title_info.loc[title_info['originalTitle'].str.contains('I Am Jonas', na=False)]
#title_info_clean.loc[title_info_clean['originalTitle'].str.contains('Along with the Gods: The Two Worlds', na=False)]
#title_info_clean.loc[((title_info_clean['originalTitle'].str.contains('Kill Chain', na=False)) & (title_info_clean['startYear'] == '2019'))]
#movies.loc[movies['Title'].str.contains('Oliver Twist', na=False)]

In [35]:
#since there are duplicate movie names with some being movies and other shows. Take the example of Oliver Twist above. 
#To exactly match the type, we also need to match the year of the movie/shows

title_info_clean_small = title_info_clean[['tconst','titleType','originalTitle','startYear','runtimeMinutes','genres']]
title_info_clean_small = title_info_clean_small[title_info_clean_small['originalTitle'].isin(all_titles)]
title_info_clean_small = title_info_clean_small.drop_duplicates(
  subset = ['originalTitle', 'startYear','titleType']).reset_index(drop = True)
title_info_clean_small = title_info_clean_small.rename(columns={"originalTitle": "Title","titleType":"type","startYear":"Year"})


In [36]:
#uncomment if you want to test match for a movie and year combination
#title= 'Terminator: Dark Fate'
#year = 2019
#title_info_clean.loc[((title_info_clean['originalTitle']== title) & (title_info_clean['startYear'].astype(int) == year)),'titleType']

In [37]:
#testing the code in the function
#uncomment for testing purposes
#testing matching with three files, IMDB file, netflix title file and Amazon title file
#Use the below title for testing titles in type_amazon file 
#title= 'Danger Close'
#title ='Terminator: Dark Fate' year = 2019
# title = 'Psycho'
# year = 2020
# runtime = np.nan
# ctry =""
# imdb_rating = np.nan
# age = np.nan
# votes =np.nan
# genres = np.nan
# old_age = '18+'

# print("title:" + title + " Year: " + str(year))
# #get type from title_info (IMDB file)
# smatch_info = title_info_clean_small.loc[((title_info_clean_small['Title']== title) & (title_info_clean_small['Year'].astype(int) == year))]
# print(smatch_info)
# #if there is a match, get type, genres, runtime minutes
# if(smatch_info.empty == False):
#     if(smatch_info['type'].empty == False):
#         stype = smatch_info['type'].values[0]
#     sconst = smatch_info['tconst'].values[0]
#     runtime = smatch_info['runtimeMinutes'].values[0]
#     genres = smatch_info['genres'].values[0]
    
#     print("match")
#     print(stype)
#     print(sconst)
    

#     #get IMDB rating and number of votes from IMDB ratings file
#     smatch_ratings = ratings.loc[(ratings['tconst']== sconst)]
    
#     #if there is a match
#     if (smatch_ratings.empty == False):
#         imdb_rating = smatch_ratings['averageRating'].values[0]
#         votes = smatch_ratings['numVotes'].values[0]
#         print(IMDB_rating)
#     #if not match, check in types_amazon and types_netflix files
# elif((type_amazon.loc[((type_amazon['title']== title) & (type_amazon['release_year'].astype(int) == year)),'type']).empty == False):
#         print("match in amazon file")
#         stype = (type_amazon.loc[((type_amazon['title']== title)),'type']).values[0].lower()
#         ctry = (type_amazon.loc[((type_amazon['title']== title)),'country']).values[0]
#         runtime = type_amazon.loc[((type_amazon['title']== title)),'duration'].values[0]
#         age = type_amazon.loc[((type_amazon['title']== title)),'rating'].values[0]
#         genres = type_amazon.loc[((type_amazon['title']== title)),'listed_in'].values[0]
#         print("In type" + stype)
#         print(runtime)
# elif((type_netflix.loc[(type_netflix['title']== title)]).empty == False):
#         print("match in netflix file")
#         print(type_netflix.loc[(type_netflix['title']== title)])
#         stype = type_netflix.loc[((type_netflix['title']== title)),'type'].values[0].lower()
#         ctry = type_netflix.loc[((type_netflix['title']== title)),'country'].values[0]
#         runtime = type_netflix.loc[((type_netflix['title']== title)),'duration'].values[0]
#         age = type_netflix.loc[((type_netflix['title']== title)),'rating'].values[0]
#         genres = type_netflix.loc[((type_netflix['title']== title)),'listed_in'].values[0]
#         print(stype)
# else:
#          print("no value")
# print(ctry)
# if(str(imdb_rating) == "" or str(genres) == "" or str(age) == "" or str(ctry) == ""):
#     print("do something")
# if(type(age) == float):
#    if(np.isnan(age) == True):
#         age = old_age
# print((stype, imdb_rating, runtime, ctry,age,votes,genres))


        
        

In [38]:
## Add additonal features number of votes, country and duration 
main['votes'] =np.nan
main['country']=np.nan
main['runtime']=np.nan
main['Age'] =np.nan
main['genres'] =np.nan

In [39]:
#prior to mapping with IMDB database, the null type values are 9515 and IMDB = 10477
main.isnull().sum()

ID                     0
Title                  0
Year                   0
Age                11395
IMDb                8441
Rotten Tomatoes        7
Netflix                0
Amazon                 0
Type                7699
votes              11395
country            11395
runtime            11395
genres             11395
dtype: int64

In [40]:
#main_small  = main.head(10)
# main_small.head()

In [41]:
#Function to set the type for all entries in the movies dataset
#title, year,oldtype, old_IMDB_rating, old_age
def get_type(movies_df):
    
    title = movies_df['Title']
    year  = movies_df['Year']
    oldtype = movies_df['Type']
    old_IMDB_rating = movies_df['IMDb']
    old_age = movies_df['Age']
    
    #print("title:" + title + " Year: " + str(year) + " Old Type:" + str(oldtype) +  " IMDB_rating:" +str(old_IMDB_rating))
    stype =""
    imdb_rating = ""
    runtime = ""
    ctry =""
    age = ""
    votes =np.nan
    genres = ""
    
    #get type from title_info (IMDB file)
    smatch_info = title_info_clean_small.loc[((title_info_clean_small['Title']== title) & (title_info_clean_small['Year'].astype(int) == year))]
    #print(smatch_info)
    
    #if there is a match, get type, genres, runtime minutes
    if(smatch_info.empty == False):

        if(smatch_info['type'].empty == False):
            stype = smatch_info['type'].values[0]
        sconst = smatch_info['tconst'].values[0]
        runtime = smatch_info['runtimeMinutes'].values[0]
        genres = smatch_info['genres'].values[0]

        #print("match")
        #print(stype)
        #print(sconst)


        #get IMDB rating and number of votes from IMDB ratings file
        smatch_ratings = ratings.loc[(ratings['tconst']== sconst)]

        #if there is a match
        if (smatch_ratings.empty == False):
            imdb_rating = smatch_ratings['averageRating'].values[0]
            votes = smatch_ratings['numVotes'].values[0]
            #print(imdb_rating)
    #if not match, check in types_amazon and types_netflix files. not matching years because of year mismatches
    elif((type_amazon.loc[((type_amazon['title']== title))]).empty == False):
        #print("match in amazon file")
        stype = (type_amazon.loc[((type_amazon['title']== title)),'type']).values[0].lower()
        ctry = (type_amazon.loc[((type_amazon['title']== title)),'country']).values[0]
        runtime = type_amazon.loc[((type_amazon['title']== title)),'duration'].values[0]
        age = type_amazon.loc[((type_amazon['title']== title)),'rating'].values[0]
        genres = type_amazon.loc[((type_amazon['title']== title)),'listed_in'].values[0]
        #print("In type" + stype)
        #print(runtime)
    elif((type_netflix.loc[((type_netflix['title']== title))]).empty == False):
        #print("match in netflix file")
        stype = type_netflix.loc[((type_netflix['title']== title)),'type'].values[0].lower()
        ctry = type_netflix.loc[((type_netflix['title']== title)),'country'].values[0]
        runtime = type_netflix.loc[((type_netflix['title']== title)),'duration'].values[0]
        age = type_netflix.loc[((type_netflix['title']== title)),'rating'].values[0]
        genres = type_netflix.loc[((type_netflix['title']== title)),'listed_in'].values[0]
        #print(stype)
    elif((main_alt.loc[((main_alt['Title']== title) & (main_alt['Year'].astype(int) == year))]).empty == False):
        #print("here")
        mmatch = main_alt.loc[((main_alt['Title']== title) & (main_alt['Year'].astype(int) == year))]
        stype = mmatch['Type'].values[0]
        imdb_rating = mmatch['IMDb'].values[0]
        age = mmatch['Age'].values[0]
        tempgenres = ast.literal_eval(mmatch['Genres'].values[0])
        genres = ",".join(tempgenres)
        #print(genres)
        #print(stype)
    else:
        stype = oldtype
    
#check if rating,runtime or genre is empty
    if(str(runtime) == "" or str(genres) == "" or str(age) == "" or str(ctry) == ""):
        #print("here")
        smatch = type_amazon.loc[((type_amazon['title']== title))]
    
        if(smatch.empty == False):
            #print("match in amazon file")
            if(str(ctry)==""):
                ctry = (type_amazon.loc[((type_amazon['title']== title)),'country']).values[0]
                if(str(runtime) ==""):
                    runtime = type_amazon.loc[((type_amazon['title']== title)),'duration'].values[0]
                if(str(age) == ""):
                    age = type_amazon.loc[((type_amazon['title']== title)),'rating'].values[0]
                if(genres == ""):
                    genres = type_amazon.loc[((type_amazon['title']== title)),'listed_in'].values[0]
        elif((type_netflix.loc[((type_netflix['title']== title))]).empty == False):
            smatch = type_netflix.loc[((type_netflix['title']== title))]
            #print("match in netflix file")
            if(smatch.empty == False):
                if(str(ctry)==""):
                    ctry = type_netflix.loc[((type_netflix['title']== title)),'country'].values[0]
                if(str(runtime) == ""):
                    runtime = type_netflix.loc[((type_netflix['title']== title)),'duration'].values[0]
                if(str(age) == ""):
                    age = type_netflix.loc[((type_netflix['title']== title)),'rating'].values[0]
                if(genres == ""):
                    genres = type_netflix.loc[((type_netflix['title']== title)),'listed_in'].values[0]
        else:
            mmatch = main_alt.loc[((main_alt['Title']== title) & (main_alt['Year'].astype(int) == year))]
            if(mmatch.empty == False):
                if(stype ==""):
                    stype = mmatch['Type'].values[0].lower()
                if(imdb_rating ==""):
                    imdb_rating = mmatch['IMDb'].values[0]
                if(str(age) == ""):
                    age = mmatch['Age'].values[0]
                if(genres == ""):
                    tempgenres = ast.literal_eval(mmatch['Genres'].values[0])
                    genres = ",".join(tempgenres)
                    #print(genres)

    if(stype == 'tv show' or stype =='tvSeries' or stype == 'tv series' or stype == 'tv mini series'):
        stype = "tvshow"
    if(stype == 'tvMovie' or stype == 'tv movie'):
        stype = 'movie'
    
    #print("IMDB new rating" + str(imdb_rating))
    #print("IMDB old rating" + str(old_IMDB_rating))
    
    if(str(imdb_rating) == ""):
#         if(np.isnan(imdb_rating) == True):
            imdb_rating = old_IMDB_rating
    
    if(str(age) ==""):
        age = old_age
        
    if(ctry == ""):
        ctry = np.nan
    if(runtime == ""):
        runtime = np.nan
    if(str(age) ==""):
        age = np.nan
    if(stype == ""):
        stype = np.nan
    if(genres == ""):
        genres = np.nan
    
    
    #print all values
    #print(stype, imdb_rating, runtime, ctry,age,votes,genres)
    
    return (stype, imdb_rating, runtime, ctry,age,votes,genres)


In [42]:
#testing on a small set
# main_small[['Type','IMDb','runtime','country','Age','votes','genres']] \
#  =main_small.apply(get_type , axis=1,result_type="expand")

In [43]:
main[['Type','IMDb','runtime','country','Age','votes','genres']] \
 =main.apply(get_type , axis=1,result_type="expand")

In [44]:
#post mapping with IMDB database, the null type values are 9515 and IMDB = 10477
main.isnull().sum()

ID                    0
Title                 0
Year                  0
Age                3009
IMDb               2568
Rotten Tomatoes       7
Netflix               0
Amazon                0
Type                110
votes              4610
country            6745
runtime            2542
genres              941
dtype: int64

In [45]:
#checking the values for movies and tvshows
main.groupby(['Type']).agg('count')

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Amazon,votes,country,runtime,genres
Type,,,,,,,,,,,,
episode,63,63,63,28,24,63,63,63,0,0,0,63
movie,7522,7522,7522,6181,5785,7515,7522,7522,4988,3160,6658,7522
music video,1,1,1,1,1,1,1,1,0,0,0,1
podcast episode,3,3,3,1,0,3,3,3,0,0,0,3
tv short,8,8,8,5,8,8,8,8,0,0,0,8
tvshow,3616,3616,3616,2107,2941,3616,3616,3616,1797,1490,2195,2675
video movie,72,72,72,34,68,72,72,72,0,0,0,72


In [46]:
main[main['Type'].isin(['video movie'])]

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Amazon,Type,votes,country,runtime,genres
1456,1457,Dragons: Dawn Of The Dragon Racers,2014,NaN,7.0,57/100,1,0,video movie,NaN,NaN,NaN,"Animation,Short,Adventure,Comedy,Family,Fantasy"
1492,1493,American Pie Presents: Girls' Rules,2020,18+,3.8,57/100,1,0,video movie,NaN,NaN,NaN,Comedy
1546,1547,Katt Williams: American Hustle,2007,18+,6.7,56/100,1,0,video movie,NaN,NaN,NaN,"Documentary,Comedy"
1955,1956,Barbie: Dolphin Magic,2017,all,5.7,52/100,1,0,video movie,NaN,NaN,NaN,"Animation,Adventure,Family,Fantasy"
2091,2092,Hulk: Where Monsters Dwell,2016,7+,5.4,51/100,1,0,video movie,NaN,NaN,NaN,"Animation,Action,Fantasy,Sci-Fi"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8548,8549,Guilty by Association,2003,18+,2.4,30/100,0,1,video movie,NaN,NaN,NaN,"Action,Crime,Drama"
8553,8554,Air Rage,2001,18+,3.2,29/100,0,1,video movie,NaN,NaN,NaN,"Action,Adventure"
8559,8560,Feeders 2: Slay Bells,1998,18+,2.5,28/100,0,1,video movie,NaN,NaN,NaN,"Comedy,Horror,Sci-Fi"
8565,8566,1313: Giant Killer Bees!,2011,16+,2.0,26/100,0,1,video movie,NaN,NaN,NaN,"Horror,Sci-Fi"


# Add emmy data to the main dataset

In [47]:
# Get a subset of the emmy dataset where titles & years are matched
emmy_1 = emmy.loc[emmy['nominee'].isin(main.Title) & emmy['year'].isin(main.Year)][['nominee', 'year', 'win']]
emmy_1.shape[0]

2403

In [48]:
# Get the number of nominations for each title in the emmy dataset
emmy_nomination_count = emmy_1.groupby(['nominee', 'year']).count()
emmy_nomination_count.rename_axis(index={'nominee': 'Title', 'year': 'Year'}, inplace=True)
emmy_nomination_count.rename(columns={'win': 'emmy_nomination'}, inplace=True)
#uncomment if you want to see the dataset
#emmy_nomination_count.sort_values('emmy_nomination', ascending=False).head(60)

In [49]:
# Add the number of nominations for each title to the main dataset
main_1 = main.set_index(['Title', 'Year']).join(emmy_nomination_count)
main_1.sort_values('emmy_nomination', ascending=False)

,,ID,Age,IMDb,Rotten Tomatoes,Netflix,Amazon,Type,votes,country,runtime,genres,emmy_nomination
Title,Year,,,,,,,,,,,,
The Pacific,2010,3611,NaN,8.3/10,83/100,0,1,tvshow,NaN,NaN,NaN,NaN,24.0
When They See Us,2019,44,TV-MA,8.9/10,86/100,1,0,tvshow,NaN,United States,1 Season,"Crime TV Shows, TV Dramas",16.0
Russian Doll,2019,90,TV-MA,7.8,82/100,1,0,tvshow,77788.0,United States,30,"Adventure,Comedy,Drama",13.0
The Night Manager,2016,3601,18+,8.1/10,86/100,0,1,tvshow,NaN,NaN,1 Season,"Action, Drama, Suspense",12.0
Genius,2017,2152,TV-14,8.3,74/100,1,1,tvshow,17936.0,India,43,"Biography,Drama,History",10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lego Set Builds Chima - Artifex,2017,5375,NaN,NaN,10/100,0,1,tvshow,NaN,NaN,NaN,NaN,NaN
The Bad Batch,2021,5385,R,8.2/10,82/100,1,0,movie,NaN,United States,119 min,"Dramas, Independent Movies, Thrillers",NaN
The Muppets,2015,5425,PG,7.4/10,64/100,1,0,movie,NaN,United States,104 min,"Children & Family Movies, Comedies",NaN


In [50]:
# Get the number of wins for each title in the emmy dataset
emmy_win_count = emmy_1.groupby(['nominee', 'year']).sum()
emmy_win_count.rename_axis(index={'nominee': 'Title', 'year': 'Year'}, inplace=True)
emmy_win_count.rename(columns={'win': 'emmy_win'}, inplace=True)
#uncomment to see the results
#emmy_win_count.sort_values('emmy_win', ascending=False).head(60)

In [51]:
# Add the number of wins for each title to the main dataset
main_2 = main_1.join(emmy_win_count)
main_2.sort_values('emmy_win', ascending=False)

,,ID,Age,IMDb,Rotten Tomatoes,Netflix,Amazon,Type,votes,country,runtime,genres,emmy_nomination,emmy_win
Title,Year,,,,,,,,,,,,,
The Pacific,2010,3611,NaN,8.3/10,83/100,0,1,tvshow,NaN,NaN,NaN,NaN,24.0,8.0
"Love, Death & Robots",2019,45,TV-MA,8.5,86/100,1,0,tvshow,128884.0,United States,15,"Action,Adventure,Animation",6.0,5.0
House of Cards,2013,28,TV-MA,8.7,88/100,1,0,tvshow,486452.0,United States,51,Drama,7.0,3.0
Queer Eye,2018,140,TV-14,8.5,78/100,1,0,tvshow,16897.0,United States,45,Reality-TV,4.0,3.0
Star Trek: Deep Space Nine,1993,105,TV-PG,8.0,80/100,1,1,tvshow,58987.0,NaN,45,"Action,Adventure,Drama",7.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lego Set Builds Chima - Artifex,2017,5375,NaN,NaN,10/100,0,1,tvshow,NaN,NaN,NaN,NaN,NaN,NaN
The Bad Batch,2021,5385,R,8.2/10,82/100,1,0,movie,NaN,United States,119 min,"Dramas, Independent Movies, Thrillers",NaN,NaN
The Muppets,2015,5425,PG,7.4/10,64/100,1,0,movie,NaN,United States,104 min,"Children & Family Movies, Comedies",NaN,NaN


# Add oscar data to the main dataset

In [52]:
# Get a subset of the emmy dataset where titles & years are matched
oscar_1 = oscar.loc[oscar.film.isin(main.Title) & oscar.year_film.isin(main.Year)][['year_film', 'film', 'winner']]
oscar_1.shape[0]

1150

In [53]:
# Get the number of nominations for each title in the oscar dataset
oscar_nomination_count = oscar_1.groupby(['film', 'year_film']).count()
oscar_nomination_count.rename_axis(index={'film': 'Title', 'year_film': 'Year'}, inplace=True)
oscar_nomination_count.rename(columns={'winner': 'oscar_nomination'}, inplace=True)
#uncomment to see results
#oscar_nomination_count.sort_values('oscar_nomination', ascending=False).head(60)

In [54]:
# Add the number of nominations for each title to the main dataset
main_3 = main_2.join(oscar_nomination_count)
main_3.sort_values('oscar_nomination', ascending=False)

,,ID,Age,IMDb,Rotten Tomatoes,Netflix,Amazon,Type,votes,country,runtime,genres,emmy_nomination,emmy_win,oscar_nomination
Title,Year,,,,,,,,,,,,,,
My Fair Lady,1964,285,G,7.8,75/100,1,0,movie,92399.0,United States,170,"Drama,Family,Musical",NaN,NaN,12.0
On the Waterfront,1954,4757,7+,8.1,79/100,0,1,movie,150189.0,NaN,108,"Crime,Drama,Thriller",NaN,NaN,12.0
Hugo,2011,174,PG,7.5,79/100,1,0,movie,315994.0,"United Kingdom, United States, France",126,"Adventure,Drama,Family",NaN,NaN,11.0
Chinatown,1974,112,18+,8.2,82/100,1,0,movie,313674.0,NaN,130,"Drama,Mystery,Thriller",NaN,NaN,11.0
The Pride of the Yankees,1942,5106,NaN,7.7,65/100,0,1,movie,10617.0,NaN,128,"Biography,Drama,Romance",NaN,NaN,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lego Set Builds Chima - Artifex,2017,5375,NaN,NaN,10/100,0,1,tvshow,NaN,NaN,NaN,NaN,NaN,NaN,NaN
The Bad Batch,2021,5385,R,8.2/10,82/100,1,0,movie,NaN,United States,119 min,"Dramas, Independent Movies, Thrillers",NaN,NaN,NaN
The Muppets,2015,5425,PG,7.4/10,64/100,1,0,movie,NaN,United States,104 min,"Children & Family Movies, Comedies",NaN,NaN,NaN


In [55]:
# Get the number of wins for each title in the emmy dataset
oscar_win_count = oscar_1.groupby(['film', 'year_film']).sum()
oscar_win_count.rename_axis(index={'film': 'Title', 'year_film': 'Year'}, inplace=True)
oscar_win_count.rename(columns={'winner': 'oscar_win'}, inplace=True)
#uncomment to see results
#oscar_win_count.sort_values('oscar_win', ascending=False).head(60)

In [56]:
# Add the number of wins for each title to the main dataset
main_4 = main_3.join(oscar_win_count)
main_4.sort_values('oscar_win', ascending=False)

,,ID,Age,IMDb,Rotten Tomatoes,Netflix,Amazon,Type,votes,country,runtime,genres,emmy_nomination,emmy_win,oscar_nomination,oscar_win
Title,Year,,,,,,,,,,,,,,,
On the Waterfront,1954,4757,7+,8.1,79/100,0,1,movie,150189.0,NaN,108,"Crime,Drama,Thriller",NaN,NaN,12.0,8.0
My Fair Lady,1964,285,G,7.8,75/100,1,0,movie,92399.0,United States,170,"Drama,Family,Musical",NaN,NaN,12.0,8.0
Patton,1970,4768,7+,7.9,77/100,0,1,movie,99630.0,NaN,172,"Biography,Drama,War",NaN,NaN,10.0,7.0
A Place in the Sun,1951,4912,13+,7.8,70/100,0,1,movie,21912.0,NaN,122,"Drama,Romance",NaN,NaN,9.0,6.0
The French Connection,1971,4775,18+,7.7,77/100,0,1,movie,117729.0,NaN,104,"Action,Crime,Drama",NaN,NaN,8.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lego Set Builds Chima - Artifex,2017,5375,NaN,NaN,10/100,0,1,tvshow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
The Bad Batch,2021,5385,R,8.2/10,82/100,1,0,movie,NaN,United States,119 min,"Dramas, Independent Movies, Thrillers",NaN,NaN,NaN,NaN
The Muppets,2015,5425,PG,7.4/10,64/100,1,0,movie,NaN,United States,104 min,"Children & Family Movies, Comedies",NaN,NaN,NaN,NaN


In [57]:
main_final = main_4.reset_index()
main_final.head()

,Title,Year,ID,Age,IMDb,Rotten Tomatoes,Netflix,Amazon,Type,votes,country,runtime,genres,emmy_nomination,emmy_win,oscar_nomination,oscar_win
0,The Irishman,2019,1,R,7.8,98/100,1,0,movie,360296.0,United States,209,"Biography,Crime,Drama",NaN,NaN,10.0,0.0
1,Dangal,2016,2,TV-PG,8.4,97/100,1,0,movie,175871.0,India,161,"Action,Biography,Drama",NaN,NaN,NaN,NaN
2,David Attenborough: A Life on Our Planet,2020,3,PG,9.0,95/100,1,0,movie,30175.0,"United States, United Kingdom",83,"Biography,Documentary",NaN,NaN,NaN,NaN
3,Lagaan: Once Upon a Time in India,2001,4,7+,8.1,94/100,1,0,movie,110453.0,NaN,224,"Drama,Musical,Sport",NaN,NaN,NaN,NaN
4,Roma,2018,5,18+,7.7,94/100,1,0,movie,151096.0,NaN,135,Drama,NaN,NaN,10.0,3.0


In [58]:
main_final.isnull().sum()

Title                   0
Year                    0
ID                      0
Age                  3009
IMDb                 2568
Rotten Tomatoes         7
Netflix                 0
Amazon                  0
Type                  110
votes                4610
country              6745
runtime              2542
genres                941
emmy_nomination     11336
emmy_win            11336
oscar_nomination    11148
oscar_win           11148
dtype: int64

In [59]:
main_final.to_csv("data/main_final.csv",index=False,encoding='utf-8')

In [60]:
main_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11395 entries, 0 to 11394
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Title             11395 non-null  object 
 1   Year              11395 non-null  int64  
 2   ID                11395 non-null  int64  
 3   Age               8386 non-null   object 
 4   IMDb              8827 non-null   object 
 5   Rotten Tomatoes   11388 non-null  object 
 6   Netflix           11395 non-null  int64  
 7   Amazon            11395 non-null  int64  
 8   Type              11285 non-null  object 
 9   votes             6785 non-null   float64
 10  country           4650 non-null   object 
 11  runtime           8853 non-null   object 
 12  genres            10454 non-null  object 
 13  emmy_nomination   59 non-null     float64
 14  emmy_win          59 non-null     float64
 15  oscar_nomination  247 non-null    float64
 16  oscar_win         247 non-null    float6

In [61]:
# Load the csv file from merge_2.ipynb to fill null values in the IMDb column
main_2 = pd.read_csv('data/main_2.csv')
final_temp = main_final.set_index(['Title', 'Year'])[['IMDb']]
main_2_temp = main_2.set_index(['Title', 'Year'])[['IMDb']]
main_final['IMDb'] = final_temp.combine_first(main_2_temp).reset_index()['IMDb']
main_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11395 entries, 0 to 11394
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Title             11395 non-null  object 
 1   Year              11395 non-null  int64  
 2   ID                11395 non-null  int64  
 3   Age               8386 non-null   object 
 4   IMDb              10258 non-null  object 
 5   Rotten Tomatoes   11388 non-null  object 
 6   Netflix           11395 non-null  int64  
 7   Amazon            11395 non-null  int64  
 8   Type              11285 non-null  object 
 9   votes             6785 non-null   float64
 10  country           4650 non-null   object 
 11  runtime           8853 non-null   object 
 12  genres            10454 non-null  object 
 13  emmy_nomination   59 non-null     float64
 14  emmy_win          59 non-null     float64
 15  oscar_nomination  247 non-null    float64
 16  oscar_win         247 non-null    float6

In [62]:
main_final.IMDb.unique()

array([7.8, 8.4, 9.0, 8.1, 7.7, 7.1, 7.6, 7.3, 8.2, 7.4, '8.2', 7.2, 8.5,
       '7.3', '8.0', 6.7, '7.6', 6.9, 6.5, 6.8, '8.5', 8.3, nan, 6.1,
       '7.4', 7.9, 7.0, '7.9', 6.6, '7.5', '7.8', 7.5, 8.0, 6.2, 6.4, 6.3,
       '7.1', '8.1', '6.8', '6.9', '7.2', 5.9, '6.2', '6.7', '7.0', 5.6,
       '5.6', '7.7', 6.0, 1.5, '5.8', '6.0', 4.2, 3.5, 5.7, '8.3', 5.8,
       5.1, '6.5', '8.4', '6.1', 5.2, 5.5, 5.3, '4.9', '6.6', '5.9',
       '6.3', 5.0, 5.4, 4.9, 4.8, 8.9, '6.4', 4.6, '5.5', 4.4, '5.3',
       '8.6', '5.7', '9.0', '8.7', 4.7, 4.5, 4.0, '4.6', 8.6, '3.3', 3.6,
       '5.2', 2.7, 4.1, 3.8, '4.2', '5.4', 4.3, '3.5', 8.8, 3.9, 2.5,
       '8.9', '5.1', 2.9, 3.7, '5.0', 8.7, 3.3, 3.4, '4.3', '4.8', '3.9',
       '3.2', '4.4', '4.1', 3.0, '4.7', 3.2, 3.1, 9.5, '3.7', '2.9', 9.1,
       '3.8', 2.8, 2.6, '3.1', '2.6', 1.8, 2.4, '4.0', '4.5', 2.1, '9.2',
       2.2, '2.0', '3.4', 1.7, '2.1', 1.9, '3.6', 9.2, '2.8', 9.8, 9.4,
       '1.5', '1.8', '2.2', '2.5', '2.4', 2.3, '1.9', 1.1, 

In [63]:
final_temp = main_final.set_index(['Title', 'Year'])[['Age']]
main_2_temp = main_2.set_index(['Title', 'Year'])[['Age']]
main_final['Age'] = final_temp.combine_first(main_2_temp).reset_index()['Age']
main_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11395 entries, 0 to 11394
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Title             11395 non-null  object 
 1   Year              11395 non-null  int64  
 2   ID                11395 non-null  int64  
 3   Age               9021 non-null   object 
 4   IMDb              10258 non-null  object 
 5   Rotten Tomatoes   11388 non-null  object 
 6   Netflix           11395 non-null  int64  
 7   Amazon            11395 non-null  int64  
 8   Type              11285 non-null  object 
 9   votes             6785 non-null   float64
 10  country           4650 non-null   object 
 11  runtime           8853 non-null   object 
 12  genres            10454 non-null  object 
 13  emmy_nomination   59 non-null     float64
 14  emmy_win          59 non-null     float64
 15  oscar_nomination  247 non-null    float64
 16  oscar_win         247 non-null    float6

In [64]:
main_final.Age.unique()

array(['R', 'TV-PG', 'PG', '7+', '18+', '13+', 'PG-13', 'TV-MA', 'TV-14',
       'TV-G', 'ALL', '16+', 'G', nan, 'NR', 'all', 'TV-Y7', 'TV-Y',
       'NC-17', 'AGES_16_', 'AGES_18_', '16', 'UNRATED', 'TV-NR'],
      dtype=object)

In [65]:
temp = main_final[['Title', 'Year', 'Age', 'IMDb', 'votes', 'country', 'runtime', 'genres']]
main_with_null = temp[temp.isnull().any(axis=1)]
main_with_null

,Title,Year,Age,IMDb,votes,country,runtime,genres
3,Lagaan: Once Upon a Time in India,2001,7+,8.1,110453.0,NaN,224,"Drama,Musical,Sport"
4,Roma,2018,18+,7.7,151096.0,NaN,135,Drama
5,To All the Boys I've Loved Before,2018,13+,7.1,99496.0,NaN,99,"Comedy,Drama,Romance"
11,Jim & Andy: The Great Beyond- Featuring a Very...,2017,18+,7.7,NaN,NaN,NaN,Documentary
14,Swades,2004,TV-PG,8.2,NaN,India,185 min,"Dramas, International Movies"
...,...,...,...,...,...,...,...,...
11389,Nature's Treasure Islands,2014,NaN,NaN,NaN,NaN,NaN,NaN
11390,Lego Set Builds Chima - Artifex,2017,NaN,NaN,NaN,NaN,NaN,NaN
11391,The Bad Batch,2021,R,8.2/10,NaN,United States,119 min,"Dramas, Independent Movies, Thrillers"
11392,The Muppets,2015,PG,7.4/10,NaN,United States,104 min,"Children & Family Movies, Comedies"


In [66]:
main_with_null['Title (Year)'] = main_with_null.Title + " (" + main_with_null.Year.astype(str) + ")"

C:\Users\kimh\AppData\Local\Temp/ipykernel_14956/2630910027.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_with_null['Title (Year)'] = main_with_null.Title + " (" + main_with_null.Year.astype(str) + ")"


In [67]:
main_with_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8593 entries, 3 to 11393
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         8593 non-null   object 
 1   Year          8593 non-null   int64  
 2   Age           6219 non-null   object 
 3   IMDb          7456 non-null   object 
 4   votes         3983 non-null   float64
 5   country       1848 non-null   object 
 6   runtime       6051 non-null   object 
 7   genres        7652 non-null   object 
 8   Title (Year)  8593 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 671.3+ KB


In [68]:
from imdb import IMDb
from tqdm import tqdm
import math

imdb = IMDb()

# columns = Age, IMDb, votes, country, runtime, genres
age_list = [] # 'certificates'
rating_list = [] # 'rating'
vote_list = [] # 'votes'
country_list = [] # 'countries'
runtime_list = [] # 'runtimes'
genre_list = [] # 'genres'

for idx, title_year in tqdm(zip(main_with_null.index, main_with_null['Title (Year)'])):
    movie_search = imdb.search_movie(title_year)
    if len(movie_search)!=0:
        movie_get = imdb.get_movie(movie_search[0].getID())
        
        try:
            if math.isnan(main_with_null.loc[idx, 'Age']):
                try:
                    main_with_null.loc[idx, 'Age'] = movie_get['certificates'][0]
#                     print(movie_get['certificates'][0], "added to", 'Age')
#                     print(main_with_null.loc[idx, 'Age'])
                except:
                    pass
        except:
            pass    
        
        try:
            if math.isnan(main_with_null.loc[idx, 'IMDb']):
                try:
                    main_with_null.loc[idx, 'IMDb'] = movie_get['rating']
#                     print(movie_get['rating'], "added to", 'IMDb')
#                     print(main_with_null.loc[idx, 'IMDb'])
                except:
                    pass
        except:
            pass
        
        try:
            if math.isnan(main_with_null.loc[idx, 'votes']):
                try:
                    main_with_null.loc[idx, 'votes'] = movie_get['votes']
#                     print(movie_get['votes'], "added to", 'votes')
#                     print(main_with_null.loc[idx, 'votes'])
                except:
                    pass
        except:
            pass
            
        try:
            if math.isnan(main_with_null.loc[idx, 'country']):
                try:
                    main_with_null.loc[idx, 'country'] = movie_get['countries'][0]
#                     print(movie_get['countries'][0], "added to", 'country')
#                     print(main_with_null.loc[idx, 'country'])
                except:
                    pass
        except:
            pass
            
        try:
            if math.isnan(main_with_null.loc[idx, 'runtime']):
                try:
                    main_with_null.loc[idx, 'runtime'] = movie_get['runtimes'][0]
#                     print(movie_get['runtimes'][0], "added to", 'runtime')
#                     print(main_with_null.loc[idx, 'runtime'])
                except:
                    pass                
        except:
            pass
        
        try:
            if math.isnan(main_with_null.loc[idx, 'genre']):
                try:
                    main_with_null.loc[idx, 'genre'] = movie_get['genres'][0]
#                     print(movie_get['genres'][0], "added to", 'genre')
#                     print(main_with_null.loc[idx, 'genre'])
                except:
                    pass                
        except:
            pass
        
#         print(idx)

0it [00:00, ?it/s]C:\Users\kimh\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
8593it [13:02:37,  5.46s/it]


In [69]:
main_with_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8593 entries, 3 to 11393
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         8593 non-null   object 
 1   Year          8593 non-null   int64  
 2   Age           7523 non-null   object 
 3   IMDb          7748 non-null   object 
 4   votes         7509 non-null   float64
 5   country       7748 non-null   object 
 6   runtime       7389 non-null   object 
 7   genres        7652 non-null   object 
 8   Title (Year)  8593 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 929.4+ KB


In [70]:
main_with_null.to_csv('main_with_null.csv', index=False, encoding='utf-8')

In [71]:
final_temp = main_final.set_index(['Title', 'Year'])
# main_with_null = main_with_null.set_index(['Title', 'Year'])
main_final_2 = final_temp.combine_first(main_with_null.set_index(['Title', 'Year'])).reset_index()
main_final_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11395 entries, 0 to 11394
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Title             11395 non-null  object 
 1   Year              11395 non-null  int64  
 2   Age               10325 non-null  object 
 3   Amazon            11395 non-null  int64  
 4   ID                11395 non-null  int64  
 5   IMDb              10550 non-null  object 
 6   Netflix           11395 non-null  int64  
 7   Rotten Tomatoes   11388 non-null  object 
 8   Title (Year)      8593 non-null   object 
 9   Type              11285 non-null  object 
 10  country           10550 non-null  object 
 11  emmy_nomination   59 non-null     float64
 12  emmy_win          59 non-null     float64
 13  genres            10454 non-null  object 
 14  oscar_nomination  247 non-null    float64
 15  oscar_win         247 non-null    float64
 16  runtime           10191 non-null  object

In [72]:
# main_final_2.to_csv('main_final_2.csv', index=False, encoding='utf-8')

In [78]:
main_final_2.genres.unique()

1113

In [79]:
# Number of unique genres in the dataset;
set([g.split(",")[0] for g in main_final_2.genres.unique() if isinstance(g, str)])

{'Action',
 'Action & Adventure',
 'Adventure',
 'Animation',
 'Anime',
 'Anime Features',
 'Anime Series',
 'Arthouse',
 'Arts',
 'Biography',
 'British TV Shows',
 'Children & Family Movies',
 'Classic & Cult TV',
 'Classic Movies',
 'Comedies',
 'Comedy',
 'Crime',
 'Crime TV Shows',
 'Cult Movies',
 'Documentaries',
 'Documentary',
 'Docuseries',
 'Drama',
 'Dramas',
 'Family',
 'Fantasy',
 'Game-Show',
 'History',
 'Horror',
 'Horror Movies',
 'Independent Movies',
 'International',
 'International Movies',
 'International TV Shows',
 'Kids',
 "Kids' TV",
 'LGBTQ',
 'Movies',
 'Music',
 'Music & Musicals',
 'Music Videos and Concerts',
 'Musical',
 'Mystery',
 'News',
 'Reality TV',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Science Fiction',
 'Short',
 'Special Interest',
 'Sport',
 'Sports',
 'Stand-Up Comedy',
 'Stand-Up Comedy & Talk Shows',
 'Suspense',
 'TV Action & Adventure',
 'TV Comedies',
 'TV Dramas',
 'TV Horror',
 'TV Shows',
 'Talk Show and Variety',
 'Talk-Show',
 'Th

In [80]:
main_final_2.runtime.unique()

array(['99 min', '92', '102 min', '104 min', '1 Season', '80', nan, '56',
       '125 min', '26', '83', '90', '9 Seasons', '61', '27', '95', '74',
       '91', '87', '82', '89', '111', '88', '169', '94', '88 min',
       '109 min', '93', '95 min', '343', '100', '60', '115', '89 min',
       '102', '85', '3 Seasons', '84', '154', '96', '98 min', '124 min',
       '114 min', '23', '130 min', '317', '86', '90 min', '30', '101',
       '149', '107 min', '158', '118 min', '144 min', '98', '78', '141',
       '135 min', '10', '107', '117', '92 min', '170', '71', '49', '22',
       '52', '116 min', '24', '105', '110', '50', '119', '96 min', '45',
       '108 min', '128', '79', '42', '103', '137', '77', '104',
       '2 Seasons', '159', '118', '144', '131 min', '43', '139', '28 min',
       '122 min', '75', '112 min', '106', '120', '94 min', '55',
       '101 min', '86 min', '136', '57', '97', '68', '28', '168 min',
       '21', '22 min', '21 min', '103 min', '73', '93 min', '113 min',
       

In [83]:
import re

In [84]:
main_final_2['votes'] = main_final_2['votes'].fillna(0).astype(int)

main_final_2['runtime'] = main_final_2['runtime'].str.replace(" min", "")
main_final_2['runtime'] = main_final_2['runtime'].apply(lambda i: np.nan if bool(re.search("Season", str(i))) else i)
main_final_2['runtime'] = main_final_2['runtime'].astype(float)

In [87]:
main_final_2.drop(['ID', 'Title (Year)'], axis=1, inplace=True)

In [88]:
main_final_2.Age.unique()

array(['TV-MA', 'Argentina:13', 'TV-G', '16+', nan, 'TV-PG',
       'United States:TV-MA', 'Australia:M', 'TV-14', '13+',
       'Australia:PG', '7+', '18+', 'Netherlands:16', 'R', 'PG',
       'AGES_18_', 'United Kingdom:U', 'PG-13', 'France:Tous publics',
       'United States:TV-14::(self-applied)', 'Germany:12', 'G',
       'Australia:MA15+', 'TV-Y7', 'Argentina:16', 'Brazil:14', 'India:U',
       'Norway:9::(TV rating)', 'all', 'United States:Not Rated', 'ALL',
       'United Kingdom:15', 'United States:TV-G', 'TV-Y',
       'Norway:15::(DVD rating)', 'Hong Kong:IIA', 'Ireland:15A',
       'Norway:A::(TV rating)', 'Germany:16::(DVD rating)', 'Finland:S',
       'Australia:G', 'United States:R', 'Canada:13+::(Quebec)',
       'Singapore:PG13', 'United States:TV-PG', 'Ireland:PG', 'NR',
       'United States:Unrated', 'Brazil:10', 'TV-NR', 'Singapore:NC16',
       'Australia:MA15+::(Netflix self-rating)', 'Mexico:C',
       'Argentina:Unrated', 'Brazil:14::(TV rating)',
       'Unit

In [89]:
main_final_2.to_csv('main_final_2.csv', index=False, encoding='utf-8')

In [90]:
main_final_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11395 entries, 0 to 11394
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Title             11395 non-null  object 
 1   Year              11395 non-null  int64  
 2   Age               10325 non-null  object 
 3   Amazon            11395 non-null  int64  
 4   IMDb              10550 non-null  object 
 5   Netflix           11395 non-null  int64  
 6   Rotten Tomatoes   11388 non-null  object 
 7   Type              11285 non-null  object 
 8   country           10550 non-null  object 
 9   emmy_nomination   59 non-null     float64
 10  emmy_win          59 non-null     float64
 11  genres            10454 non-null  object 
 12  oscar_nomination  247 non-null    float64
 13  oscar_win         247 non-null    float64
 14  runtime           9408 non-null   float64
 15  votes             11395 non-null  int32  
dtypes: float64(5), int32(1), int64(3), objec

 # Other general tests

In [ ]:
type_netflix.isnull().sum()

In [ ]:
type_amazon.groupby("type").agg("count")

In [ ]:
movies[movies['Title'] == 'Psycho']

In [ ]:
#list of titles that got tagged as tvshows from the movies dataset
#main_5[((main_5['Title'].isin(movie_names)) & (main_5['Type'] == 'tvshow'))]

In [ ]:
main_final.loc[main_final['Type'].isnull()]

In [ ]:
null_types = main_final.loc[main_final['Type'].isnull(),'Title'].unique()

In [ ]:
main_final[main_final['Amazon']==1].groupby(['Amazon','Type']).agg('count')

In [ ]:
main_final[main_final['Netflix']==1].groupby(['Netflix','Type']).agg('count')

In [ ]:
main_alt.head()

In [ ]:
main_alt.nunique(axis=0)

In [ ]:
main_alt.isnull().sum()

In [ ]:
main_alt.size

In [ ]:
main_alt[main_alt['Title'].isin(null_types)]

In [ ]:
main_alt[main_alt['Amazon']==1].groupby(['Amazon','Type']).agg('count')

In [ ]:
main_alt[main_alt['Netflix']==1].groupby(['Netflix','Type']).agg('count')

In [ ]:
main_alt.groupby(['Netflix','Amazon']).agg('count')

In [ ]:
main_alt.groupby(['Type']).agg('count')

In [ ]:
 main_alt.loc[main['Type'].isnull(),'Title'].unique()

In [ ]:
#number of rows
main_alt.shape[0]

In [ ]:
#number of rows
main.shape[0]

In [ ]:
#https://en.wikipedia.org/wiki/Motion_Picture_Association_film_rating_system
#what is 7+? and what is the difference between 18+ and 16+
main_alt.Age.unique()

In [ ]:
type_amazon.groupby(['type']).agg('count')

In [ ]:
main_alt[main_alt['emmy_nomination'] > 0]

In [ ]:
emmy[emmy['nominee']=='My Brother\'s Keeper']

In [ ]:
movies[movies['Title']=='Virunga']